# Introduction

## Motivation

The COVID-19 pandemic is a challenging time for all of us. AI-models in NLP could help process COVID-19 information in medical interviews more interpretable and visual. This research will focus on symptom extraction from medical dialogue before and after COVID-19.  

## RQs

What are the most common symptoms before and after COVID-19 in medical interviews? 

## Example as a task illustration

The code is designed to work as follows:
- Feed it a medical dialogue, where symptoms are discussed
- The code will extract the symptoms
- The code will display the most common symptoms in that conversation

An example can be illustrated with the following conversation: \
Patient: Hello doctor, these last few days I have been __coughing__ a lot. \
Doctor: That is unfortunate to hear, do you have any other symptoms like __sore throat__, __chest pains__, etc? \
Patient: I have a __cough__ and in addition to that also a __sore throat__, but that's about it. \
Doctor: Alright, a __cough__ and a __sore throat__ are symptoms of the Coronavirus but because of the time of year I assume you just have a __cold__. It is advised to take a test and take medicine for your __cough__ and __sore throat__. 

The model would then output 'cough' and 'sore throat' as most common symtpoms discussed in this dialogue. 

## Related literature
__Transformers__: A transformer model is used in Natural Language Processing (NLP) for sequence-to-sequence tasks such as language translation (Vaswani et al., 2017). Transformer models apply an attention mechanism, where the model determines which parts of an input sequence are important. This results in understanding relationships between all words in a sentence, without having to worry about their respective position. Transformers consist of an encoder that maps the input sequence into a higher dimensional space and a decoder that produces an output sequence, for example a translation of the input sequence. Because the encoder reads the whole input sequence at once, instead of reading the input from left to right or right to left, it is considered bidirectional. 

__BERT__: BERT (Bidirectional Encoder Representations from Transformers) is a language representation model that applies the bidirectional training of Transformers to language modelling (Devlin et al., 2018). Because of the bidirectionality, the model learns the context of a word based on all of its surroundings, instead of just the previous words. This allows for a deeper understanding of words and sentences. Using the fact that BERT can capture a deeper understanding of words and sentences, it can be applied to multiple NLP tasks by fine-tuning the model by adding a new layer on top of the model. The architecture of the new layer depends on the tasks one wants to do. 


__RoBERTa__: RoBERTa is an extension of the BERT model, it is trained longer and on more data, providing a stronger core model (Liu et al., 2019). For NLP tasks, adding an additional layer on top is still possible. 

__NER__: NER (Named Entity Recognition) tries to classify named entities mentioned in text to certain categories such as person, location, disease or symptom. It is explained before that a new layer is needed for the BERT or roBERTa model to perform NLP tasks. In the case of NER, this additional layer is a randomly initialized classifier (Vamvas, 2019). After training, this classifier outputs the most likely category of the words in the input sequence.  

# Experimental Setup

T-NER (https://github.com/asahi417/tner) is a python tool for language model finetuning on NER. It provides pre-trained RoBERTa models, including the pre-trained RoBERTa bc5cdr model. The bc5cdr dataset (Wei et al., 2015) consists of 1500 PubMed articles that are annotated for biomedical usage (chemicals, diseases, etc.). 

I wouldn't recommend running the installations in your terminal, because inconsistencies could arise. Therefore, please create a new conda environment (to prevent inconsistencies) and run: \
<code>
pip install pip -U \
pip install sentencepiece \
pip install sortedcontainers==2.1.0 \
pip install tner 
</code> 
    
Additionaly you will need to have pandas and nltk installed:

<code>
pip install pandas
pip install nltk
</code>

In [3]:
# Imports
import RQ1_utils as RQ1
import os
import pandas as pd
from tner import TransformersNER
from pprint import pprint

[nltk_data] Downloading package punkt to /home/dilaratank/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


If the T-NER import doesn't work try to do the following:
- Please open jupyter notebook in your terminal from the place where you did the installations. So if you installed the packages at '/home/user/Desktop', please navigate there in your terminal and open the notebook from there (this worked for me).
- If that doesn't work please run the code for RQ1 seperately in this google colab notebook:

<a href="https://colab.research.google.com/github/dilaratank/roBERTa-Symptom-Tracking/blob/main/Symptom_Tracking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Datasets

The following datasets will be used: 
- COVID-19 Dialogue Datase (during/after covid) https://www.kaggle.com/xuehaihe/covid-dialogue-dataset?select=COVID-Dialogue-Dataset-English.txt
- MedDialog Dataset (English) (before covid) https://github.com/UCSD-AI4H/Medical-Dialogue-System
  - This dataset consists of 4 datasets, the 'icliniq' dataset will be used for this project because of the wide variety of medical subjects

These datasets are structured as follows: 
- ID number
- Description
- Dialogue


### Dataset Preprocessing

The roBERTa model requires the data to be structured per sentence, which is done in the preprocessing steps. First, the data is split on dialogue to get rid of other unneccessary information. Thereafter, the data is split on sentences and saved as a .csv file for later use.

Preprocessing the data might take a while! Especially the icliniq dialogue.

In [2]:
if not os.path.exists('RQ1/saved/COVID-Dialogue-Dataset-English.csv'):
    RQ1.preprocess_to_csv('RQ1/saved/COVID-Dialogue-Dataset-English.txt', 'RQ1/saved')
if not os.path.exists('RQ1/saved/icliniq_dialogue.csv'):
    RQ1.preprocess_to_csv('RQ1/saved/icliniq_dialogue.txt', 'RQ1/saved')

# Note: If you actually want to run the code, uncomment and run the following:
#RQ1.preprocess_to_csv('RQ1/saved/COVID-Dialogue-Dataset-English.txt', 'RQ1/saved')
#RQ1.preprocess_to_csv('RQ1/saved/icliniq_dialogue.txt', 'RQ1/saved')

The data now looks like this:

In [3]:
covid_dialogue_df = pd.read_csv('RQ1/saved/COVID-Dialogue-Dataset-English.csv')
icliniq_dialogue_df = pd.read_csv('RQ1/saved/icliniq_dialogue.csv')
covid_dialogue_df.head()

,sentences
0,"Hello doctor, I get a cough for the last few d..."
1,No raise in temperature but feeling tired with...
2,No contact with any Covid-19 persons.
3,It has been four to five days and has drunk a ...
4,Doctors have shut the OP so do not know what t...


Because the Icliniq dialogue data is 60 times larger than the COVID-19 dialogue data, it will be shortened to +/ 5500 sentences. For this purpose a saved .csv file will be used so that the results will not change (due to randomness). To run this code please remove '`if not os.path.exists(...)` etc'. If you run this code, do note that the dataframe will be different each time. 

In [4]:
if not os.path.exists('RQ1/saved/icliniq_dialogue_short.csv'):
    np.random.seed(10)
    dfcopy = icliniq_dialogue_df.copy()
    remove_n = 263500
    drop_indices = np.random.choice(dfcopy.index, remove_n, replace=False)
    icliniq_dialogue_short_df = dfcopy.drop(drop_indices)
    icliniq_dialogue_short_df.to_csv('data_preprocessed_rq1/icliniq_dialogue_short.csv')
icliniq_dialogue_short_df = pd.read_csv('RQ1/saved/icliniq_dialogue_short.csv')

## Aproaches

- Use pre-trained roBERTa to extract symptoms from medical dialogue before and after COVID-19
- Display most common symtoms
- Model evaluation using accuracy score


## Implementation details

First, let's download the pretrained model from HuggingFace. If this doesn't work, please open this code in a google colab. 

In [5]:
trainer = TransformersNER('asahi417/tner-xlm-roberta-base-bc5cdr')

2021-03-04 16:48:56 INFO     *** initialize network ***
/home/dilaratank/anaconda3/envs/tner-env/lib/python3.9/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


For the task of determining the most used symptoms in a healhcare dialogue, the function `get_symptoms_data(df)` has been designed. It consists of 5 steps that will be discussed. The steps are also commented in the function for easier understanding and following. 

While looping over the seperate sentences in the dataframe: \
__Step 1__: First, the prediction of the sentence is extracted using the function `trainer.predict()` that is part of the T-NER package. \
__Step 2__: Second, only the predicted symptoms are extracted using the costum function `get_predicted_symptoms(prediction)`. This function returns only the sympoms that are mentioned in the sentence by only selecting the predicted entities that are labelled 'disease'. \
__Step 3__: Then, all these symptoms are gathered and put in a different dataframe. Note that this layout is not structured by sentence. All the symptoms are placed seperately in the dataframe. \
__Step 4__: After that, a counter is applied to this dataframe. This allows to extract the top 5 used symptoms. \
__Step 5__: The top 5 used symptoms are extracted and plotted for easy observation. 

In [6]:
def get_predicted_symptoms(prediction):
    """
    This function takes in the prediction of a sentence of the pre-trained model 
    and returns the symptoms mentioned in that sentence.
    """
    
    symptoms = []
    
    # Check if there is a predicted entity
    if len(prediction[0]['entity']) > 0:
        number_of_entities = len(prediction[0]['entity'])
    
        # Loop over predicted entities and get symptoms (here called: disease)
        for i in range(number_of_entities):
            if prediction[0]['entity'][i]['type'] == 'disease':
                symptoms.append(prediction[0]['entity'][i]['mention'])
            
    return symptoms 

def get_symptoms_data(df):
    """
    This function takes a (preprocessed) dataframe as input, determines the predicted
    symptoms per phrase, and outputs a dictionary with the 5 most frequent symptoms used,
    while also plotting the counts of the words in a barplot. 
    
    The steps (1-5) are explained in the report. 
    """
    
    all_symptoms = []
    skipped_sentences = 0
    
    for ind in tqdm(df.index):
        sentence = df['sentences'][ind]
        
        # Padding is needed because algorithms is not used to small sentences
        if len(sentence) < 45:
            sentence = sentence+'...'
        
        # Step 1
        try:
            prediction = trainer.predict([sentence])
        except:
            skipped_sentences += 1
            continue
        
        # Step 2
        predicted_symptom = get_predicted_symptoms(prediction)
        predicted_symptom = [x.lower() for x in predicted_symptom]
        predicted_symptom = [x.split(', ')[0] for x in predicted_symptom]
        
        # Step 3
        for symptom in predicted_symptom:
            if symptom != 'coronavirus': # Remove 'coronavirus', because it is not a symptom
                all_symptoms.append(symptom)
            
    df_symptoms = DataFrame(all_symptoms,columns=['symptoms'])
    
    # Step 4
    count_symptoms = Counter()
    
    # Split on 'enter(\n)' so that grouped words stay 1 symptom
    df_symptoms['symptoms'].str.lower().str.split('\n').apply(count_symptoms.update) 

    # Most common symptoms: Step 5
    print('Skipped sentences: ', skipped_sentences)
    top5 = dict(count_symptoms.most_common(5))
    print(top5)
    plt.bar(range(len(top5)), list(top5.values()), align='center')
    plt.xticks(range(len(top5)), list(top5.keys()))
    plt.title('Most common symptoms')
    plt.show()
        

# Evaluation

## Metrics
The model will be evaluated using the accuracy metric. The T-NER package also included evaluation functions, but the accuracy function that is used for this research has ben designed from scratch. The reason for this is because the T-NER functions needed the data to be in a different format. It has been decided that pre-processing the code again, only for that purpose, was not optimal. The accuracy function that has been developed keeps track of the total number of symptoms and the number of correctly predicted sympoms. The number of correctly predicted symptoms are then divided by the total number of symptoms, which gives the accuracy score. 

In [4]:
covid_dialogue_ann_df = pd.read_csv('RQ1/annotated_data/COVID-Dialogue-Dataset-English-annotated.csv')
icliniq_dialogue_ann_df = pd.read_csv('RQ1/annotated_data/icliniq_dialogue_nodup_annotated.csv')

The evaluation data is annotated by hand. Per sentence, the symptoms in that sentence are added to another column in the same row of the dataframe. It looks like this:

In [5]:
covid_dialogue_ann_df.head()

,sentences,symptoms
0,"Hello doctor, I get a cough for the last few d...",cough
1,No raise in temperature but feeling tired with...,"raise in temperature, tired"
2,No contact with any Covid-19 persons.,NaN
3,It has been four to five days and has drunk a ...,NaN
4,Doctors have shut the OP so do not know what t...,NaN


This will take approximately 1 or 2 minutes per dataset.

In [10]:
print('-- COVID-19 Dialogue --')
RQ1.accuracy(covid_dialogue_ann_df, trainer)
print()
print('-- Icliniq Dialogue --')
RQ1.accuracy(icliniq_dialogue_ann_df, trainer)

  0%|          | 0/507 [00:00<?, ?it/s]

-- COVID-19 Dialogue --


  0%|          | 1/500 [00:00<01:26,  5.77it/s]

Ground truth symptoms:  303
Correctly predicted symptoms  239
accuracy:  0.7887788778877888

-- Icliniq Dialogue --


100%|██████████| 500/500 [01:44<00:00,  4.77it/s]

Ground truth symptoms:  201
Correctly predicted symptoms  167
accuracy:  0.8308457711442786


## Results

The results for COVID-19 dialogue: 

![](RQ1/most_common_symptoms_covid.png)

In [ ]:
# Note: to run the actual code, please uncomment and run:
#get_symptoms_data(covid_dialogue_df)

The results for Icliniq dialogue:

![](RQ1/most_common_symptoms_icliniq.png)

In [ ]:
# Note: to run the actual code, please uncomment and run:
#get_symptoms_data(icliniq_dialogue_short_df)

## Error analysis

# Findings

## Illustration

In the first plot (on the left) you can see the results for the COVID-19 dialogue again, and in the second plot (on the right) you can see the results for the Icliniq dialogue again. It can be seen that both results include the symptoms 'fever' and 'infection'. 

<table><tr><td><img src='RQ1/most_common_symptoms_covid.png'></td><td><img src='RQ1/most_common_symptoms_icliniq.png'></td></tr></table>

## Interpretation

The results show that pneumonia, fever, cough, sore throat and infection are the most common symptoms that patients and doctors talk about in COVID-19 related medical interviews. This seems valid, because they all actually are symptoms of COVID-19. Pneumonia is the most used symptom here, whereas before COVID-19 the most used symtpoms was pain. This is also seems valid, because in very general interviews (with general topics) pain can be used to describe a variety of areas. Furthermore, before COVID-19 the most used symptoms inclide anxiety, bleeding, fever and infection. 

It is clear that fever and infection were already important symptoms in medical interviews and with COVID-19 they stayed important, especially fever was more and more discussed. Furthermore, pain, anxiety and bleeding made way for pneumonia, cough and sore throat. 

## Discussion

The results can, of course, not be generalized to all medical interviews because just a sub-set of data is used for this research. However, it is still insightful to know the changes in medical dialogue before and after COVID-19. 

# Conclusions

## Summary

This research can be summarized as follows. To determine the most common symptoms in a medical dialogue before and after COVID-19 a pre-trained RoBERTa model is used to extract symptoms from a given sequence of text. These symptoms are then collected and counted, whereafter the top 5 most common sympoms are extracted and displayed. 

For this research it can be stated that the most common symptoms in COVID-19 symptoms dialogue are pneumonia, fever, cough, sore throat and infection. Whereas that used to be pain, anxiety, bleeding, fever and infection. Even though the results might not be generalized to all medical interviews out there, symptom-tracking could still be a useful tool for summarizing medical interviews or quickly displaying common symptoms for a certain disease. 

## Lessons learned

# References

Chih-Hsuan  Wei,  Yifan  Peng,  Robert  Leaman,  Al-lan  Peter  Davis,   Carolyn  J.  Mattingly,   Jiao  Li,Thomas   C.   Wiegers,    and   Zhiyong   Lu.   2015.Overview  of  the  BioCreative  V  Chemical  DiseaseRelation (CDR) Task.   InProceedings of the FifthBioCreative Challenge Evaluation Workshop. pages154–166.

Devlin, J., Chang, M. W., Lee, K., & Toutanova, K. (2018). Bert: Pre-training of deep 
bidirectional transformers for language understanding. arXiv preprint arXiv:1810.04805.

Liu, Y., Ott, M., Goyal, N., Du, J., Joshi, M., Chen, D., ... & Stoyanov, V. (2019). Roberta: A robustly optimized bert pretraining approach. arXiv preprint arXiv:1907.11692.

Vamvas, J. (2019). BERT for NER. Von https://vamvas.ch/bert-for-ner.

Vaswani, A., Shazeer, N., Parmar, N., Uszkoreit, J., Jones, L., Gomez, A. N., ... & Polosukhin, I. (2017). Attention is all you need. arXiv preprint arXiv:1706.03762.